In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端机器学习: MLOps阶段6：使用Vertex AI Prediction开始使用AutoML文本模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_text_model_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_text_model_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_automl_text_model_online.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>        
</table>
<br/><br/><br/>

## 概览

本教程演示了如何从`AutoML`文本模型进行在线预测。

### 目标

在本教程中，您将学习如何使用 `Vertex AI Prediction` 与 `AutoML` 文本模型。

本教程使用以下 Google Cloud ML 服务和资源:

- `Vertex AutoML`
- `Vertex AI Prediction`
- `Vertex AI Datasets`
- `Vertex AI Models`
- `Vertex AI Endpoints`

执行的步骤包括:

- 创建一个 Vertex `Dataset` 资源。
- 训练一个 `AutoML` 模型。
- 将模型部署到一个 `Endpoint` 资源。
- 进行在线预测。

### 数据集

本教程使用的数据集是来自[Kaggle Datasets](https://www.kaggle.com/ritresearch/happydb)的[Happy Moments dataset](https://www.kaggle.com/ritresearch/happydb)。在本教程中使用的数据集版本存储在一个公共云存储桶中。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您预计的使用情况生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Vertex AI Workbench笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

云存储指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简洁的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端shell中运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端shell中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install tensorflow $USER_FLAG -q

### 重新启动内核

安装了附加包之后，需要重新启动笔记本内核，这样它才能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行时

本教程不需要 GPU 运行时。

### 设置您的 Google Cloud 项目

**以下步骤是必需的，无论您使用的是哪种笔记本环境。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您会获得 $300 的免费信用，用于支付计算/存储成本。

2. [确保为您的项目启用了计费功能。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、Compute Engine API 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您正在本地运行此笔记本，您需要安装 [Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 运行以 `!` 开头的行作为 shell 命令，并会插值以 `$` 开头的 Python 变量。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以尝试使用 `gcloud` 来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于本笔记本的其余操作。以下是Vertex AI支持的区域。 我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶进行Vertex AI的训练。并非所有区域都支持所有Vertex AI服务。

了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在参加现场教程会话，您可能正在使用共享测试帐户或项目。为了避免用户之间在创建的资源上产生名称冲突，您需要为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的谷歌云帐户

**如果您正在使用 Vertex AI Workbench 笔记本**，您的环境已经通过认证。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行认证您的帐户的OAuth。

**否则**，请按照以下步骤操作：

在云控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐户对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，选择**Storage Object Admin**。

点击创建。包含您密钥的 JSON 文件将下载到您的本地环境。

在下面的单元格中将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的笔记本环境如何，以下步骤都是必需的。**

当您初始化用于 Python 的 Vertex AI SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是您数据集和模型资源相关数据在会话之间保留的地方。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有 Google 云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶还不存在的情况下才能执行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容验证访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

### 初始化Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像和分配给虚拟机（VM）实例的GPU数量。例如，要使用一个带有4个Nvidia Telsa K80 GPU分配给每个VM的GPU容器映像，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，可以指定`(None, None)`来使用一个可以在CPU上运行的容器映像。

了解有关[您地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)的更多信息。

了解有关[按机型的GPU兼容性](https://cloud.google.com/vertex-ai/docs/training/configure-compute#gpu-compatibility-table)的更多信息。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置要用于预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96\]个数

*注：您也可以在训练和部署时使用n2和e2机器类型，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

现在你可以开始创建你自己的AutoML文本分类模型。

云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/NL-classification/happiness.csv"

## 快速浏览您的数据

本教程使用存储在公共云存储桶中的快乐时刻数据集的版本，使用CSV索引文件。

首先快速浏览数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`TextDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，以将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标注模式。

此操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.TextDataset.create(
    display_name="Happy Moments" + "_" + UUID,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
)

print(dataset.resource_name)

### 创建并运行训练管道

要训练自动化机器学习模型，需要执行两个步骤：1）创建训练管道，2）运行管道。

#### 创建训练管道

使用`AutoMLTextTrainingJob`类创建一个自动化机器学习训练管道，包括以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：为模型训练指定任务类型。
  - `classification`：文本分类模型。
  - `sentiment`：文本情感分析模型。
  - `extraction`：文本实体提取模型。
- `multi_label`：如果是一个分类任务，指定是单标签（False）还是多标签（True）。
- `sentiment_max`：如果是一个情感分析任务，指定情感值的最大值。

实例化的对象是训练管道的有向无环图（DAG）。

In [ ]:
dag = aiplatform.AutoMLTextTrainingJob(
    display_name="happydb_" + UUID, prediction_type="classification"
)

print(dag)

#### 运行训练管道

接下来，您可以通过调用`run`方法来运行DAG以启动训练作业，方法参数如下：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留置数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。

当`run`方法完成时，将返回`Model`资源。

训练管道的执行可能需要长达4小时。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="happydb_" + UUID,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

## 创建 `Endpoint` 资源

您可以使用 `Endpoint.create()` 方法来创建一个 `Endpoint` 资源。至少，您需要指定 endpoint 的显示名称。可选地，您可以指定项目和位置（区域）；否则，设置将继承您使用 `init()` 方法初始化 Vertex AI SDK 时设置的值。

在这个例子中，指定了以下参数：

- `display_name`：`Endpoint` 资源的人类可读名称。
- `project`：您的项目 ID。
- `location`：您的区域。
- `labels`：（可选）对 `Endpoint` 的用户定义的元数据，格式为键/值对。

此方法返回一个 `Endpoint` 对象。

了解有关[Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)的更多信息。

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name="happydb_" + UUID,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

## 部署 `Model` 资源到一个 `Endpoint` 资源。

您可以将一个或多个 `Vertex AI Model` 资源实例部署到同一个端点。每个部署的 `Vertex AI Model` 资源都有自己的提供二进制服务的部署容器。

*注意:* 对于这个示例，您的 `AutoML` 模型的部署容器是预定义的。

### 扩展

`Vertex AI Endpoint` 资源支持三种类型的扩展：

- 无扩展：提供二进制服务的部署到单个 VM 实例。
- 手动扩展：提供二进制服务的部署到固定数量的多个 VM 实例。
- 自动扩展：根据负载的不同，提供二进制服务的部署到不同数量的 VM 实例。

对于 AutoML 模型，仅支持无扩展和手动扩展。

### 手动扩展

在下一个示例中，您将部署`Vertex AI模型`资源到`Vertex AI端点`资源以进行手动扩展 -- 一定数量（大于1）的VM实例。换句话说，当模型部署时，将预配固定数量的VM实例，并且在模型取消部署之前将保持预配状态。

在此示例中，您将使用指定参数的最小量部署模型，如下所示：

- `model`：要部署的`Model`资源。
- `machine_type`：每个VM实例的机器类型。
- `deployed_model_displayed_name`：部署模型实例的可读名称。
- `min_replica_count`：要预配的最小VM实例（节点）数。
- `max_replica_count`：要预配的最大VM实例（节点）数。
- `accelerator_type`：每个VM实例的硬件加速器类型，如有的话。
- `accelerator_count`：每个VM实例的硬件加速器数量，如有的话。

对于手动扩展，将在模型部署期间预配固定数量的VM实例。

*注意：* 对于手动扩展，节点的最小和最大数量被设置为相同的值。

In [ ]:
MIN_NODES = 2
MAX_NODES = 2

if DEPLOY_NGPU:
    response = endpoint.deploy(
        model=model,
        deployed_model_display_name="happydb_" + UUID,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
    )
else:
    response = endpoint.deploy(
        model=model,
        deployed_model_display_name="bank_" + UUID,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

发送在线预测请求

向您部署的模型发送在线预测。

获取测试项目

您可以从数据集中随机选择一个示例作为测试项目。

In [ ]:
test_item = ! gsutil cat $IMPORT_FILE | head -n1
if len(test_item[0]) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

### 进行预测

现在您的 `Model` 资源已部署到一个 `Endpoint` 资源，您可以通过向 `Endpoint` 资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式为:

     { 'content': text_string }

由于 predict() 方法可以接受多个条目（实例），请将您的单个测试项目作为一个测试项目的列表发送。

#### 响应

来自 predict() 调用的响应是一个包含以下条目的 Python 字典：

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `displayNames`: 每个类标签的类名。
- `confidences`: 每个类别标签的预测置信度，介于 0 和 1 之间。
- `deployed_model_id`: 执行预测的已部署 `Model` 资源的 Vertex AI 标识符。

In [ ]:
instances_list = [{"content": test_item}]

prediction = endpoint.predict(instances_list)
print(prediction)

#### 下线模型

当您完成预测工作后，您需要将模型从`Vertex AI 终端节点`资源中下线。这将取消所有计算资源，并停止对已部署模型的计费。

In [ ]:
endpoint.undeploy_all()

删除模型

接下来，您可以使用`delete（）`方法删除`Vertex AI Model`资源。

In [ ]:
model.delete()

#### 删除数据集

接下来，您可以使用`delete（）`方法删除`Vertex AI数据集`资源。

In [ ]:
dataset.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除您用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
delete_bucket = True

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}